最终版

In [2]:
import pulp

# 创建问题实例
prob = pulp.LpProblem("Wire_Production_ILP", pulp.LpMinimize)

# 决策变量定义
x1 = pulp.LpVariable('x1', cat='Binary')  # 是否新购II型拉丝机
x2 = pulp.LpVariable('x2', cat='Binary')  # 是否改造塑包机
x3 = pulp.LpVariable('x3', cat='Binary')  # 是否新购联合机

# 拉丝机生产时间
a1I = pulp.LpVariable('a1I', lowBound=0)
a2I = pulp.LpVariable('a2I', lowBound=0)
a1II = pulp.LpVariable('a1II', lowBound=0)
a2II = pulp.LpVariable('a2II', lowBound=0)

# 塑包机生产时间
b1I = pulp.LpVariable('b1I', lowBound=0)
b2I = pulp.LpVariable('b2I', lowBound=0)
b1II = pulp.LpVariable('b1II', lowBound=0)
b2II = pulp.LpVariable('b2II', lowBound=0)

# 联合机生产时间
c1 = pulp.LpVariable('c1', lowBound=0)
c2 = pulp.LpVariable('c2', lowBound=0)

# 成本函数组成部分
depreciation = 10000 * (1 * x1 + 0.5 * x2 + 2.5 * x3)
annual_fee = 10000 * (3 + 5 * x1 + 8 + 2 * x2 + 14 * x3)
operation_fee = (
    5 * (a1I + a2I)
    + 7 * (a1II + a2II)
    + 8 * (b1I + b2I + b1II + b2II)
    + 12 * (c1 + c2)
)
waste_loss_a = 0.6 * a1I + 0.48 * a2I + 0.9 * a1II + 0.84 * a2II
waste_loss_b = 1.8 * b1I + 1.5 * b2I + 2.4 * b1II + 1.95 * b2II
waste_loss_c = 2.4 * c1 + 1.8 * c2

# 目标函数
prob += depreciation + annual_fee + operation_fee + waste_loss_a + waste_loss_b + waste_loss_c

# 约束1：市场需求
prob += 0.98 * (a1I + 1.5 * a1II) == 3000 + 1.2 * b1I + 1.6 * b1II
prob += 0.98 * (a2I + 1.4 * a2II) == 2000 + 1.0 * b2I + 1.3 * b2II
prob += 0.97 * (1.2 * b1I + 1.6 * b1II + 1.6 * c1) == 10000
prob += 0.97 * (1.0 * b2I + 1.3 * b2II + 1.2 * c2) == 8000

# 约束2：设备工作时长限制
prob += a1I + a2I <= 8000
prob += a1II + a2II <= 8000 * x1
prob += b1I + b2I <= 8000 * (1 - x2)
prob += b1II + b2II <= 8000 * x2
prob += c1 + c2 <= 8000 * x3

# 求解问题
prob.solve()

# 输出结果
print("Status:", pulp.LpStatus[prob.status])
for v in prob.variables():
    print(f"{v.name} = {v.varValue}")
print("Total Cost = ", pulp.value(prob.objective))


Status: Optimal
a1I = 0.0
a1II = 2040.8163
a2I = 8000.0
a2II = 393.13516
b1I = 0.0
b1II = 0.0
b2I = 6379.3814
b2II = 0.0
c1 = 6443.299
c2 = 1556.701
x1 = 1.0
x2 = 0.0
x3 = 1.0
Total Cost =  572914.7311244


错误版

未化简为线性规划

In [1]:
import pulp

# 创建问题实例
prob = pulp.LpProblem("Integer_Mixed_LP", pulp.LpMinimize)

# 定义变量
x1 = pulp.LpVariable('x1', cat='Binary')
x2 = pulp.LpVariable('x2', cat='Binary')
x3 = pulp.LpVariable('x3', cat='Binary')

a1I = pulp.LpVariable('a1I', lowBound=0)
a2I = pulp.LpVariable('a2I', lowBound=0)
a1II = pulp.LpVariable('a1II', lowBound=0)
a2II = pulp.LpVariable('a2II', lowBound=0)
b1 = pulp.LpVariable('b1', lowBound=0)
b2 = pulp.LpVariable('b2', lowBound=0)
c1 = pulp.LpVariable('c1', lowBound=0)
c2 = pulp.LpVariable('c2', lowBound=0)

# 构造目标函数
depreciation = 10000 * (1 * x1 + 0.5 * x2 + 2.5 * x3)
annual_fee = 10000 * (3 + 5 * x1 + 8 + 2 * x2 + 14 * x3)
operation_fee = 5 * (a1I + a2I) + 7 * (a1II + a2II) + 8 * (b1 + b2) + 12 * (c1 + c2)
waste_loss_a = 0.6 * a1I + 0.48 * a2I + 0.9 * a1II + 0.84 * a2II
waste_loss_b = (1.8 + 0.6 * x2) * b1 + (1.5 + 0.45 * x2) * b2
waste_loss_c = 2.4 * c1 + 1.8 * c2

prob += (
    depreciation
    + annual_fee
    + operation_fee
    + waste_loss_a
    + waste_loss_b
    + waste_loss_c
)

# 添加约束
# 市场需求
prob += 0.98 * (a1I + 1.5 * a1II) == 3000 + (1.2 + 0.4 * x2) * b1
prob += 0.98 * (a2I + 1.4 * a2II) == 2000 + (1 + 0.3 * x2) * b2
prob += 0.97 * ((1.2 + 0.4 * x2) * b1 + 1.6 * c1) == 10000
prob += 0.97 * ((1 + 0.3 * x2) * b2 + 1.2 * c2) == 8000

# 设备工作约束
prob += a1I + a2I <= 8000
prob += a1II + a2II <= 8000 * x1
prob += b1 + b2 <= 8000
prob += c1 + c2 <= 8000 * x3

# 求解
prob.solve()

# 输出结果
print("Status:", pulp.LpStatus[prob.status])
for v in prob.variables():
    print(f"{v.name} = {v.varValue}")
print("Total Cost = ", pulp.value(prob.objective))


TypeError: Non-constant expressions cannot be multiplied